<a href="https://colab.research.google.com/github/karino2/tegashiki/blob/master/TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install tensorflow==2.0.0-beta0

In [0]:
# !pip install tf-nightly

In [0]:
# today tensorflow seems update and batchnormalization name scope is changed. I downgrade for a while
# !pip install tensorflow==1.13.1

In [0]:
import tensorflow as tf

In [0]:
# from tensorflow.contrib import lite

In [3]:
print(tf.VERSION)
# print(tf.__version__)

1.14.0-rc1


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
tf.keras.backend.clear_session()
tf.keras.backend.set_learning_phase(0) 

In [0]:
# MODEL_DIR="gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/1560480405/"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_trainflaghandle/1560556738/"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/export_expgen_convencdec_posfc/1560866662"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed/1560903469"
# MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/1560910035"
MODEL_DIR="gs://karino2-tegashiki/models/exports/expgen_convencdec_myembed2/1560933034"

In [0]:
LITE = tf.lite
# LITE = lite

In [0]:
# !gsutil ls {MODLE_DIR}

### Convert to TFLite

In [0]:
MAX_STROKE_NUM=13
MAX_ONE_STROKE_LEN=50
INPUT_TYPE_DIM=3

In [8]:
# converter = LITE.TFLiteConverter.from_saved_model(MODEL_DIR, input_shapes={"stroke_input" : [MAX_STROKE_NUM, MAX_ONE_STROKE_LEN, INPUT_TYPE_DIM]})
converter = LITE.TFLiteConverter.from_saved_model(MODEL_DIR)
# workaround https://github.com/tensorflow/tensorflow/issues/26413
# converter.post_training_quantize=True
converter.optimizations = [LITE.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

W0619 08:33:21.670366 140165457979264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/convert_saved_model.py:60: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0619 08:33:23.154832 140165457979264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0619 08:33:24.570284 140165457979264 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/util.

357832

In [0]:
!mv converted_model.tflite convencdec_myembed2.tflite

In [11]:
!gsutil cp convencdec_myembed2.tflite gs://karino2-tegashiki/models/mobile/

Copying file://convencdec_myembed2.tflite [Content-Type=application/octet-stream]...
/ [1 files][349.5 KiB/349.5 KiB]                                                
Operation completed over 1 objects/349.5 KiB.                                    


In [12]:
!gsutil ls gs://karino2-tegashiki/models/mobile/

gs://karino2-tegashiki/models/mobile/convencdec_myembed2.tflite


In [10]:
!ls -l *.tflite

-rw-r--r-- 1 root root 357832 Jun 19 08:33 convencdec_myembed2.tflite


In [0]:
patch_file = "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py"

In [0]:
import os

In [0]:
org_file = patch_file + "_org"

In [0]:
if not os.path.exists(org_file):
  !cp {patch_file} {org_file}

In [0]:
%pycat {patch_file}

In [0]:
%%writefile {patch_file}


In [0]:
%%writefile {patch_file}
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Helpers to manipulate a tensor graph in python.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import copy
import re
import six

from tensorflow.core.framework import attr_value_pb2
from tensorflow.core.framework import graph_pb2
from tensorflow.core.framework import node_def_pb2
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.util import deprecation
from tensorflow.python.util.tf_export import tf_export

_VARIABLE_OPS = {
    "Assign",
    "AssignAdd",
    "AssignSub",
    "Queue",
    "ScatterAdd",
    "ScatterSub",
    "ScatterUpdate",
    "TruncatedNormal",
    "Variable",
    "VariableV2",
}


def _is_variable_op(op):
  """Returns true if 'op' refers to a Variable node."""
  return op in _VARIABLE_OPS


@deprecation.deprecated(
    date=None,
    instructions="Use `tf.compat.v1.graph_util.must_run_on_cpu`")
@tf_export(v1=["graph_util.must_run_on_cpu"])
def must_run_on_cpu(node, pin_variables_on_cpu=False):
  """Returns True if the given node_def must run on CPU, otherwise False.

  Args:
    node: The node to be assigned to a device. Could be either an ops.Operation
      or NodeDef.
    pin_variables_on_cpu: If True, this function will return False if node_def
      represents a variable-related op.

  Returns:
    True if the given node must run on CPU, otherwise False.
  """

  if isinstance(node, ops.Operation):
    node_def = node.node_def
  else:
    assert isinstance(node, node_def_pb2.NodeDef)
    node_def = node

  # If the op is a variable-related op, should we pin it on CPU?
  if pin_variables_on_cpu and _is_variable_op(node_def.op):
    return True

  # Constant operations producing a string or int32 must run on CPU.
  if node_def.op == "Const":
    # Get the value of the 'dtype' attr
    dtype = node_def.attr["dtype"].type
    if dtype == dtypes.string or dtype == dtypes.int32:
      return True

  if node_def.op in ["DynamicStitch", "ParallelDynamicStitch"]:
    dtype = node_def.attr["T"].type
    if dtype == dtypes.int32:
      # DynamicStitch on GPU only works for int32 values.
      return True

  if node_def.op in ["Cast"]:
    dtype = node_def.attr["SrcT"].type
    if dtype == dtypes.int32:
      # Cast on GPU does not works for int32 values.
      return True
  return False


################################################################################
#
# device functions for use in with g.device(...)
#
################################################################################


def _node_name(n):
  if n.startswith("^"):
    return n[1:]
  else:
    return n.split(":")[0]


def _extract_graph_summary(graph_def):
  """Extracts useful information from the graph and returns them."""
  name_to_input_name = {}  # Keyed by the dest node name.
  name_to_node = {}  # Keyed by node name.

  # Keeps track of node sequences. It is important to still output the
  # operations in the original order.
  name_to_seq_num = {}  # Keyed by node name.
  seq = 0
  for node in graph_def.node:
    n = _node_name(node.name)
    name_to_node[n] = node
    name_to_input_name[n] = [_node_name(x) for x in node.input]
    if "_class" in node.attr:
      # Prevent colocated nodes being lost
      for v in node.attr["_class"].list.s:
        v_str = v.decode("utf-8")
        if v_str.startswith("loc:@"):
          colocated_node = v_str[5:]
          name_to_input_name[n].append(colocated_node)
    name_to_seq_num[n] = seq
    seq += 1
  return name_to_input_name, name_to_node, name_to_seq_num


def _assert_nodes_are_present(name_to_node, nodes):
  """Assert that nodes are present in the graph."""
  for d in nodes:
    assert d in name_to_node, "%s is not in graph" % d


def _bfs_for_reachable_nodes(target_nodes, name_to_input_name):
  """Breadth first search for reachable nodes from target nodes."""
  nodes_to_keep = set()
  # Breadth first search to find all the nodes that we should keep.
  next_to_visit = target_nodes[:]
  while next_to_visit:
    node = next_to_visit[0]
    del next_to_visit[0]
    if node in nodes_to_keep:
      # Already visited this node.
      continue
    nodes_to_keep.add(node)
    if node in name_to_input_name:
      next_to_visit += name_to_input_name[node]
  return nodes_to_keep


@deprecation.deprecated(
    date=None,
    instructions="Use `tf.compat.v1.graph_util.extract_sub_graph`")
@tf_export(v1=["graph_util.extract_sub_graph"])
def extract_sub_graph(graph_def, dest_nodes):
  """Extract the subgraph that can reach any of the nodes in 'dest_nodes'.

  Args:
    graph_def: A graph_pb2.GraphDef proto.
    dest_nodes: A list of strings specifying the destination node names.
  Returns:
    The GraphDef of the sub-graph.

  Raises:
    TypeError: If 'graph_def' is not a graph_pb2.GraphDef proto.
  """

  if not isinstance(graph_def, graph_pb2.GraphDef):
    raise TypeError("graph_def must be a graph_pb2.GraphDef proto.")

  if isinstance(dest_nodes, six.string_types):
    raise TypeError("dest_nodes must be a list.")

  name_to_input_name, name_to_node, name_to_seq_num = _extract_graph_summary(
      graph_def)
  _assert_nodes_are_present(name_to_node, dest_nodes)

  nodes_to_keep = _bfs_for_reachable_nodes(dest_nodes, name_to_input_name)

  nodes_to_keep_list = sorted(
      list(nodes_to_keep), key=lambda n: name_to_seq_num[n])
  # Now construct the output GraphDef
  out = graph_pb2.GraphDef()
  for n in nodes_to_keep_list:
    out.node.extend([copy.deepcopy(name_to_node[n])])
  out.library.CopyFrom(graph_def.library)
  out.versions.CopyFrom(graph_def.versions)

  return out


@deprecation.deprecated(
    date=None,
    instructions="Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`"
)
@tf_export(v1=["graph_util.tensor_shape_from_node_def_name"])
def tensor_shape_from_node_def_name(graph, input_name):
  """Convenience function to get a shape from a NodeDef's input string."""
  # To get a tensor, the name must be in the form <input>:<port>, for example
  # 'Mul:0'. The GraphDef input strings don't always have the port specified
  # though, so if there isn't a colon we need to add a default ':0' to the end.
  if ":" not in input_name:
    canonical_name = input_name + ":0"
  else:
    canonical_name = input_name
  tensor = graph.get_tensor_by_name(canonical_name)
  shape = tensor.get_shape()
  return shape


@deprecation.deprecated(
    date=None,
    instructions="Use `tf.compat.v1.graph_util.convert_variables_to_constants`")
@tf_export(v1=["graph_util.convert_variables_to_constants"])
def convert_variables_to_constants(sess,
                                   input_graph_def,
                                   output_node_names,
                                   variable_names_whitelist=None,
                                   variable_names_blacklist=None):
  """Replaces all the variables in a graph with constants of the same values.

  If you have a trained graph containing Variable ops, it can be convenient to
  convert them all to Const ops holding the same values. This makes it possible
  to describe the network fully with a single GraphDef file, and allows the
  removal of a lot of ops related to loading and saving the variables.

  Args:
    sess: Active TensorFlow session containing the variables.
    input_graph_def: GraphDef object holding the network.
    output_node_names: List of name strings for the result nodes of the graph.
    variable_names_whitelist: The set of variable names to convert (by default,
                              all variables are converted).
    variable_names_blacklist: The set of variable names to omit converting
                              to constants.

  Returns:
    GraphDef containing a simplified version of the original.
  """

  def trace_back_find_variable(origin_name, name_to_nodes):

    nodes_in_path = set()
    control_ops = ["Enter", "Exit", "NextIteration", "Switch"]

    current_name = origin_name
    while name_to_nodes[current_name].op != "VarHandleOp":
      nodes_in_path.add(current_name)
      current_node = name_to_nodes[current_name]
      op_name = current_node.op
      if op_name in control_ops or op_name == "Identity" :
        curr_input_name = _node_name(current_node.input[0])
      else:
        raise ValueError("Op type %s should not be in the path " +
                         "between ReadVariableOp and VarHandleOp" % current_node.op)
      current_name = curr_input_name

    return current_name, nodes_in_path


  def create_const_op(node_name, dtype, data, data_shape=None):
    """Creates a Const op."""
    output_node = node_def_pb2.NodeDef()
    output_node.op = "Const"
    output_node.name = node_name
    output_node.attr["dtype"].CopyFrom(dtype)
    output_node.attr["value"].CopyFrom(
        attr_value_pb2.AttrValue(
            tensor=tensor_util.make_tensor_proto(
                data, dtype=dtype.type, shape=data_shape)))
    return output_node

  # This graph only includes the nodes needed to evaluate the output nodes, and
  # removes unneeded nodes like those involved in saving and assignment.
  inference_graph = extract_sub_graph(input_graph_def, output_node_names)

  # Identify the ops in the graph.
  map_name_to_node = {
      node.name: node for node in inference_graph.node
  }

  # Get list of variables.
  variable_names = []
  variable_dict_names = []
  resource_identity_types = {}
  read_variable_op_types = {}
  for node in inference_graph.node:
    if node.op in ["Variable", "VariableV2", "VarHandleOp"]:
      variable_name = node.name
      if ((variable_names_whitelist is not None and
           variable_name not in variable_names_whitelist) or
          (variable_names_blacklist is not None and
           variable_name in variable_names_blacklist)):
        continue
      variable_dict_names.append(variable_name)
      if node.op == "VarHandleOp":
        variable_names.append(variable_name + "/Read/ReadVariableOp:0")
      else:
        variable_names.append(variable_name + ":0")
    elif node.op in ["ReadVariableOp", "ResourceGather", "VariableShape"]:
      # There can be one or more Identity or control flow ops in between the ReadVariableOp and
      # VarHandleOp.  Store them with the associated dtypes.
      source_op_name, nodes_in_path = trace_back_find_variable(_node_name(node.input[0]), map_name_to_node)
      dtype = map_name_to_node[source_op_name].attr["dtype"]
      for node_name in nodes_in_path:
        resource_identity_types[node_name] = dtype
      read_variable_op_types[node.name] = dtype

  # Gets map of variables and the associated data.
  if variable_names:
    returned_variables = sess.run(variable_names)
  else:
    returned_variables = []
  variables_data_map = dict(zip(variable_dict_names, returned_variables))
  logging.info("Froze %d variables.", len(returned_variables))

  # Reconstruct the graph with constants in place of variables.
  output_graph_def = graph_pb2.GraphDef()
  how_many_converted = 0
  for input_node in inference_graph.node:
    output_node = node_def_pb2.NodeDef()
    if input_node.name in variables_data_map:
      data = variables_data_map[input_node.name]
      output_node = create_const_op(input_node.name, input_node.attr["dtype"],
                                    data, data.shape)
      how_many_converted += 1
    elif input_node.name in resource_identity_types:
      # Converts the Identities of type RESOURCE_DT to the appropriate type
      # based on the input they are referencing.
      output_node.CopyFrom(input_node)
      output_node.attr["T"].CopyFrom(resource_identity_types[input_node.name])
    elif input_node.op == "ReadVariableOp":
      # The first branch converts all VarHandleOps of ResourceVariables to
      # constants, so we need to convert the associated ReadVariableOps to
      # Identity ops.
      output_node.op = "Identity"
      output_node.name = input_node.name
      output_node.input.extend([input_node.input[0]])
      output_node.attr["T"].CopyFrom(input_node.attr["dtype"])
      if "_class" in input_node.attr:
        output_node.attr["_class"].CopyFrom(input_node.attr["_class"])
    elif input_node.op == "ResourceGather":
      # The first branch converts all VarHandleOps of ResourceGather to
      # constants, so we need to convert the associated ResourceGather to Gather
      # ops with a Const axis feeding into it.
      if input_node.attr["batch_dims"].i != 0:
        raise ValueError("batch_dims != 0 is not supported by freeze_graph.")
      axis_data = input_node.attr["batch_dims"].i
      axis_node_name = input_node.name + "/axis"
      axis_dtype = input_node.attr["Tindices"]
      output_axis_node = create_const_op(axis_node_name, axis_dtype, axis_data)
      output_graph_def.node.extend([output_axis_node])

      output_node.op = "GatherV2"
      output_node.name = input_node.name
      output_node.input.extend(
          [input_node.input[0], input_node.input[1], axis_node_name])
      output_node.attr["Tparams"].CopyFrom(input_node.attr["dtype"])
      output_node.attr["Tindices"].CopyFrom(input_node.attr["Tindices"])
      output_node.attr["Taxis"].CopyFrom(axis_dtype)
      if "_class" in input_node.attr:
        output_node.attr["_class"].CopyFrom(input_node.attr["_class"])
    elif input_node.op == "VariableShape":
      output_node.op = "Shape"
      output_node.name = input_node.name
      output_node.input.extend([input_node.input[0]])
      output_node.attr["T"].CopyFrom(read_variable_op_types[input_node.name])
      output_node.attr["out_type"].CopyFrom(input_node.attr["out_type"])
    else:
      output_node.CopyFrom(input_node)
    output_graph_def.node.extend([output_node])

  output_graph_def.library.CopyFrom(inference_graph.library)
  logging.info("Converted %d variables to const ops.", how_many_converted)
  return output_graph_def


@deprecation.deprecated(
    date=None,
    instructions="Use `tf.compat.v1.graph_util.remove_training_nodes`")
@tf_export(v1=["graph_util.remove_training_nodes"])
def remove_training_nodes(input_graph, protected_nodes=None):
  """Prunes out nodes that aren't needed for inference.

  There are nodes like Identity and CheckNumerics that are only useful
  during training, and can be removed in graphs that will be used for
  nothing but inference. Here we identify and remove them, returning an
  equivalent graph. To be specific, CheckNumerics nodes are always removed, and
  Identity nodes that aren't involved in control edges are spliced out so that
  their input and outputs are directly connected.

  Args:
    input_graph: Model to analyze and prune.
    protected_nodes: An optional list of names of nodes to be kept
      unconditionally. This is for example useful to preserve Identity output
      nodes.

  Returns:
    A list of nodes with the unnecessary ones removed.
  """
  if not protected_nodes:
    protected_nodes = []

  types_to_remove = {"CheckNumerics": True}

  input_nodes = input_graph.node
  names_to_remove = {}
  for node in input_nodes:
    if node.op in types_to_remove and node.name not in protected_nodes:
      names_to_remove[node.name] = True

  nodes_after_removal = []
  for node in input_nodes:
    if node.name in names_to_remove:
      continue
    new_node = node_def_pb2.NodeDef()
    new_node.CopyFrom(node)
    input_before_removal = node.input
    del new_node.input[:]
    for full_input_name in input_before_removal:
      input_name = re.sub(r"^\^", "", full_input_name)
      if input_name in names_to_remove:
        continue
      new_node.input.append(full_input_name)
    nodes_after_removal.append(new_node)

  types_to_splice = {"Identity": True}
  control_input_names = set()
  node_names_with_control_input = set()
  for node in nodes_after_removal:
    for node_input in node.input:
      if "^" in node_input:
        control_input_names.add(node_input.replace("^", ""))
        node_names_with_control_input.add(node.name)

  names_to_splice = {}
  for node in nodes_after_removal:
    if node.op in types_to_splice and node.name not in protected_nodes:
      # We don't want to remove nodes that have control edge inputs, because
      # they might be involved in subtle dependency issues that removing them
      # will jeopardize.
      if node.name not in node_names_with_control_input:
        names_to_splice[node.name] = node.input[0]

  # We also don't want to remove nodes which are used as control edge inputs.
  names_to_splice = {name: value for name, value in names_to_splice.items()
                     if name not in control_input_names}

  nodes_after_splicing = []
  for node in nodes_after_removal:
    if node.name in names_to_splice:
      continue
    new_node = node_def_pb2.NodeDef()
    new_node.CopyFrom(node)
    input_before_removal = node.input
    del new_node.input[:]
    for full_input_name in input_before_removal:
      input_name = re.sub(r"^\^", "", full_input_name)
      while input_name in names_to_splice:
        full_input_name = names_to_splice[input_name]
        input_name = re.sub(r"^\^", "", full_input_name)
      new_node.input.append(full_input_name)
    nodes_after_splicing.append(new_node)

  output_graph = graph_pb2.GraphDef()
  output_graph.node.extend(nodes_after_splicing)
  return output_graph

Overwriting /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/graph_util_impl.py


In [0]:
patch_fname = os.path.basename(patch_file)

In [0]:
!cp {patch_file} ./{patch_fname}

In [0]:
!echo ./{patch_fname}

./graph_util_impl.py


In [0]:
%%writefile {temp}

Hello, Hy This is test.
"""triple quote here."""

Overwriting test.txt


In [0]:
target_content = """
# Copyright 2018 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Functions to convert SavedModel to frozen GraphDefs."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.lite.python.convert import tensor_name
from tensorflow.core.framework import types_pb2
from tensorflow.python.client import session
from tensorflow.python.framework import graph_util as tf_graph_util
from tensorflow.python.framework import ops
from tensorflow.python.platform import tf_logging as logging
from tensorflow.python.saved_model import constants
from tensorflow.python.saved_model import loader


def _log_tensor_details(tensor_info):
  """Log tensor details: name, shape, and type."""
  for key in tensor_info:
    val = tensor_info[key]
    dtype = types_pb2.DataType.Name(val.dtype)
    if val.tensor_shape.unknown_rank:
      shape = "unknown_rank"
    else:
      dims = [str(dim.size) for dim in val.tensor_shape.dim]
      shape = "({})".format(", ".join(dims))

    logging.info("Tensor's key in saved_model's tensor_map: %s", key)
    logging.info(" tensor name: %s, shape: %s, type: %s", val.name, shape,
                 dtype)


def get_meta_graph_def(saved_model_dir, tag_set):
  """Validate saved_model and extract MetaGraphDef.

  Args:
    saved_model_dir: saved_model path to convert.
    tag_set: Set of tag(s) of the MetaGraphDef to load.

  Returns:
    The meta_graph_def used for tflite conversion.

  Raises:
    ValueError: No valid MetaGraphDef for given tag_set.
  """
  with session.Session(graph=ops.Graph()) as sess:
    return loader.load(sess, tag_set, saved_model_dir)


def get_signature_def(meta_graph, signature_key):
  """Get the signature def from meta_graph with given signature_key.

  Args:
    meta_graph: meta_graph_def.
    signature_key: signature_def in the meta_graph_def.

  Returns:
    The signature_def used for tflite conversion.

  Raises:
    ValueError: Given signature_key is not valid for this meta_graph.
  """
  signature_def_map = meta_graph.signature_def
  signature_def_keys = set(signature_def_map.keys())
  logging.info(
      "The given SavedModel MetaGraphDef contains SignatureDefs with the "
      "following keys: %s", signature_def_keys)
  if signature_key not in signature_def_keys:
    raise ValueError("No '{}' in the SavedModel\'s SignatureDefs. Possible "
                     "values are '{}'.".format(signature_key,
                                               ",".join(signature_def_keys)))
  return signature_def_map[signature_key]


def get_inputs_outputs(signature_def):
  """Get inputs and outputs from SignatureDef.

  Args:
    signature_def: SignatureDef in the meta_graph_def for conversion.

  Returns:
    The inputs and outputs in the graph for conversion.
  """
  inputs_tensor_info = signature_def.inputs
  outputs_tensor_info = signature_def.outputs
  logging.info("input tensors info: ")
  _log_tensor_details(inputs_tensor_info)
  logging.info("output tensors info: ")
  _log_tensor_details(outputs_tensor_info)

  def gather_names(tensor_info):
    return [tensor_info[key].name for key in tensor_info]

  inputs = gather_names(inputs_tensor_info)
  outputs = gather_names(outputs_tensor_info)
  return inputs, outputs


def _get_tensors(graph, signature_def_tensor_names=None,
                 user_tensor_names=None):
  """Gets the tensors associated with the tensor names.

  Either signature_def_tensor_names or user_tensor_names should be provided. If
  the user provides tensors, the tensors associated with the user provided
  tensor names are provided. Otherwise, the tensors associated with the names in
  the SignatureDef are provided.

  Args:
    graph: GraphDef representing graph.
    signature_def_tensor_names: Tensor names stored in either the inputs or
      outputs of a SignatureDef. (default None)
    user_tensor_names: Tensor names provided by the user. (default None)

  Returns:
    List of tensors.

  Raises:
    ValueError:
      signature_def_tensors and user_tensor_names are undefined or empty.
      user_tensor_names are not valid.
  """
  tensors = []
  if user_tensor_names:
    # Sort the tensor names.
    user_tensor_names = sorted(user_tensor_names)

    tensors = get_tensors_from_tensor_names(graph, user_tensor_names)
  elif signature_def_tensor_names:
    tensors = [
        graph.get_tensor_by_name(name)
        for name in sorted(signature_def_tensor_names)
    ]
  else:
    # Throw ValueError if signature_def_tensors and user_tensor_names are both
    # either undefined or empty.
    raise ValueError(
        "Specify either signature_def_tensor_names or user_tensor_names")

  return tensors


def get_tensors_from_tensor_names(graph, tensor_names):
  """Gets the Tensors associated with the `tensor_names` in the provided graph.

  Args:
    graph: TensorFlow Graph.
    tensor_names: List of strings that represent names of tensors in the graph.

  Returns:
    A list of Tensor objects in the same order the names are provided.

  Raises:
    ValueError:
      tensor_names contains an invalid tensor name.
  """
  # Get the list of all of the tensors.
  tensor_name_to_tensor = {
      tensor_name(tensor): tensor for op in graph.get_operations()
      for tensor in op.values()
  }

  # Get the tensors associated with tensor_names.
  tensors = []
  invalid_tensors = []
  for name in tensor_names:
    tensor = tensor_name_to_tensor.get(name)
    if tensor is None:
      invalid_tensors.append(name)
    else:
      tensors.append(tensor)

  # Throw ValueError if any user input names are not valid tensors.
  if invalid_tensors:
    raise ValueError("Invalid tensors '{}' were found.".format(
        ",".join(invalid_tensors)))
  return tensors


def set_tensor_shapes(tensors, shapes):
  """Sets Tensor shape for each tensor if the shape is defined.

  Args:
    tensors: TensorFlow ops.Tensor.
    shapes: Dict of strings representing input tensor names to list of
      integers representing input shapes (e.g., {"foo": : [1, 16, 16, 3]}).

  Raises:
    ValueError:
      `shapes` contains an invalid tensor.
      `shapes` contains an invalid shape for a valid tensor.
  """
  if shapes:
    tensor_names_to_tensor = {tensor_name(tensor): tensor for tensor in tensors}
    for name, shape in shapes.items():
      if name not in tensor_names_to_tensor:
        raise ValueError("Invalid tensor \'{}\' found in tensor shapes "
                         "map.".format(name))
      if shape is not None:
        tensor = tensor_names_to_tensor[name]
        try:
          tensor.set_shape(shape)
        except ValueError as error:
          message = ("The shape of tensor '{0}' cannot be changed from {1} to "
                     "{2}. {3}".format(name, tensor.get_shape(), shape,
                                       str(error)))
          raise ValueError(message)


def freeze_saved_model(saved_model_dir, input_arrays, input_shapes,
                       output_arrays, tag_set, signature_key):
  """Converts a SavedModel to a frozen graph.

  Args:
    saved_model_dir: SavedModel directory to convert.
    input_arrays: List of input tensors to freeze graph with. Uses input arrays
      from SignatureDef when none are provided.
    input_shapes: Dict of strings representing input tensor names to list of
      integers representing input shapes (e.g., {"foo": : [1, 16, 16, 3]}).
      Automatically determined when input shapes is None (e.g., {"foo" : None}).
    output_arrays: List of output tensors to freeze graph with. Uses output
      arrays from SignatureDef when none are provided.
    tag_set: Set of tags identifying the MetaGraphDef within the SavedModel to
      analyze. All tags in the tag set must be present.
    signature_key: Key identifying SignatureDef containing inputs and outputs.

  Returns:
    frozen_graph_def: Frozen GraphDef.
    in_tensors: List of input tensors for the graph.
    out_tensors: List of output tensors for the graph.

  Raises:
    ValueError:
      SavedModel doesn't contain a MetaGraphDef identified by tag_set.
      signature_key is not in the MetaGraphDef.
      assets/ directory is in the MetaGraphDef.
      input_shapes does not match the length of input_arrays.
      input_arrays or output_arrays are not valid.
  """
  # Read SignatureDef.
  meta_graph = get_meta_graph_def(saved_model_dir, tag_set)
  signature_def = get_signature_def(meta_graph, signature_key)
  inputs, outputs = get_inputs_outputs(signature_def)

  # Check SavedModel for assets directory.
  collection_def = meta_graph.collection_def
  if constants.ASSETS_KEY in collection_def:
    raise ValueError("SavedModels with assets/ directory are not supported.")

  graph = ops.Graph()
  with session.Session(graph=graph) as sess:
    loader.load(sess, meta_graph.meta_info_def.tags, saved_model_dir)

    # Gets input and output tensors.
    # TODO(zhixianyan): Use TFLite supported Op list to filter outputs.
    in_tensors = _get_tensors(graph, inputs, input_arrays)
    out_tensors = _get_tensors(graph, outputs, output_arrays)
    set_tensor_shapes(in_tensors, input_shapes)

    output_names = [node.split(":")[0] for node in outputs]
    frozen_graph_def = tf_graph_util.convert_variables_to_constants(
        sess, graph.as_graph_def(), output_names)

    return frozen_graph_def, in_tensors, out_tensors
"""

In [0]:
help(tf.lite.TFLiteConverter)

Help on class TFLiteConverter in module tensorflow.lite.python.lite:

class TFLiteConverter(builtins.object)
 |  Convert a TensorFlow model into `output_format` using TOCO.
 |  
 |  This is used to convert from a TensorFlow GraphDef or SavedModel into either a
 |  TFLite FlatBuffer or graph visualization.
 |  
 |  Attributes:
 |  
 |    inference_type: Target data type of real-number arrays in the output file.
 |      Must be `{tf.float32, tf.uint8}`. (default tf.float32)
 |    inference_input_type: Target data type of real-number input arrays. Allows
 |      for a different type for input arrays in the case of quantization.
 |      Must be `{tf.float32, tf.uint8}`. (default `inference_type`)
 |    output_format: Output file format. Currently must be `{TFLITE,
 |      GRAPHVIZ_DOT}`. (default TFLITE)
 |    quantized_input_stats: Dict of strings representing input tensor names
 |      mapped to tuple of floats representing the mean and standard deviation
 |      of the training data (e.

In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.post_training_quantize=True
tflite_quantized_model=converter.convert()
open("quantized_model.tflite", "wb").write(tflite_quantized_model)

In [0]:
!gsutil ls {MODEL_DIR}

gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/
gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/1560480405/
gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/temp-b'1560480340'/
gs://karino2-tegashiki/models/exports/export_expgen_rnn_small_dropout05_fixshape/temp-b'1560480363'/
